In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pandas as pd
from tqdm import tqdm

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)

In [3]:
# Параметры
test_path = 'fv2_test.parquet'  
model_path = '2_DCN_MLP.pth'  
test_output_path = '2_predictions.csv'  
BATCH_SIZE = 8192
data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/'  

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class MyDataset(Dataset):
    def __init__(self, interactions, device):
        self.device = device
        self.users = torch.tensor(interactions['user_id'].values, dtype=torch.long, device=self.device)
        self.items = torch.tensor(interactions['item_id'].values, dtype=torch.long, device=self.device)
        self.ages = torch.tensor(interactions['age'].values, dtype=torch.long, device=self.device)
        self.item_durations = torch.tensor(interactions['item_duration'].values, dtype=torch.long, device=self.device)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx]

In [6]:
class DCN(nn.Module):
    def __init__(self, input_dim, num_cross_layers):
        super(DCN, self).__init__()
        self.input_dim = input_dim
        self.num_cross_layers = num_cross_layers
        
        self.cross_weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, 1)) for _ in range(num_cross_layers)]
        )
        self.cross_biases = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim)) for _ in range(num_cross_layers)]
        )
        
    def forward(self, x):
        x0 = x
        for i in range(self.num_cross_layers):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x

class DCNWithMLP(nn.Module):
    def __init__(self, input_dim, num_cross_layers=3, hidden_dim=128, output_dim=3):
        super(DCNWithMLP, self).__init__()

        self.batch_norm = nn.BatchNorm1d(input_dim)

        self.dcn = DCN(input_dim, num_cross_layers)

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.batch_norm(x)
        
        x = self.dcn(x)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

In [7]:
test = pd.read_parquet(f'{data_folder}{test_path}', engine='pyarrow')

In [8]:
test_ds = MyDataset(test, device)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
input_dim = len(test.columns) 
num_cross_layers = 3 
model = DCNWithMLP(input_dim=input_dim, num_cross_layers=num_cross_layers).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()  

C:\Users\Николай\AppData\Local\Temp\ipykernel_10636\3371622768.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


DCNWithMLP(
  (batch_norm): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dcn): DCN(
    (cross_weights): ParameterList(
        (0): Parameter containing: [torch.float32 of size 4x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 4x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 4x1 (cuda:0)]
    )
    (cross_biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 4 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 4 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 4 (cuda:0)]
    )
  )
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [23]:
import torch.nn.functional as F
predictions = []
with torch.no_grad():
    with tqdm(test_dl, desc="Evaluating Test Data", unit="batch") as t:
        for users, items, ages, item_durations in t:
            inputs = torch.cat((users.unsqueeze(1), items.unsqueeze(1), ages.unsqueeze(1), item_durations.unsqueeze(1)), dim=1).float()
            
            outputs = model(inputs)
            
            probs = F.softmax(outputs, dim=1)
            
            weighted_sum = (probs * torch.arange(probs.size(1), device=probs.device)).sum(dim=1)
            
            predictions.extend(weighted_sum.cpu().numpy())

Evaluating Test Data: 100%|██████████| 203/203 [00:22<00:00,  9.17batch/s]


In [24]:
# Сохранение предсказаний
test['prediction'] = predictions
test[['user_id', 'item_id', 'prediction']].to_csv(f'{data_folder}{test_output_path}', index=False)
print(f"Предсказания сохранены в {test_output_path}")

Предсказания сохранены в 2_predictions.csv
